In [386]:
from lusidtools.jupyter_tools import toggle_code

In [387]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.8062.0


In [388]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolio_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [389]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"

# 1. Create Portfolio

In [390]:
portfolio_code = "BondPortfolioForPnLCalc"

try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

# 2. Create Bond

In [391]:
def create_bond(
    currency, 
    payment_frequency,
    roll_convention, 
    day_count_convention, 
    payment_calendars, 
    reset_calendars, 
    settle_days, 
    reset_days, 
    start_date, 
    maturity_date, 
    dom_ccy, 
    principal, 
    coupon_rate, 
    bond_identifier,
    bond_name
):

    flow_conventions = lm.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention, 
        payment_calendars=payment_calendars, 
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = lm.Bond(
        start_date=start_date, 
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal, 
        coupon_rate=coupon_rate, 
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond"
    )

    # define the instrument to be upserted
    bond_definition = lm.InstrumentDefinition(
        name=bond_name, 
        identifiers={'ClientInternal': lm.InstrumentIdValue(bond_identifier)},
        definition=bond
    )

    # upsert the instrument
    upsert_request = {bond_identifier: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[bond_identifier].lusid_instrument_id
    print(bond_luid)

# 2.1 Upsert a bond

In [409]:
currency="USD"
payment_frequency="6M"
roll_convention="MF"
day_count_convention="ActAct"
payment_calendars=[]
reset_calendars=[]
settle_days=0
reset_days=0
start_date=datetime(2019, 2, 15, 00, tzinfo=pytz.utc)
maturity_date=datetime(2049, 2, 15, 00, tzinfo=pytz.utc)
dom_ccy="USD"
principal=1
coupon_rate=0.03
bond_identifier='T_3_02/15/49'
bond_name='T 3 02/15/49'

create_bond(
    currency, 
    payment_frequency,
    roll_convention, 
    day_count_convention, 
    payment_calendars, 
    reset_calendars, 
    settle_days, 
    reset_days, 
    start_date, 
    maturity_date, 
    dom_ccy, 
    principal, 
    coupon_rate, 
    bond_identifier,
    bond_name
)


LUID_0000BYLX


# 3. Transactions

## 3.1 Create Transaction Request

In [393]:
transactions = pd.read_csv('data/bond_transaction_data.csv')
transactions

txn_id     type     client_id            trade_date       settlement_date  \
0  txn001  StockIn  T_3_02/15/49  2019-04-19T10:00:00Z  2019-04-20T10:00:00Z   

   quantity  price  total_consideration currency                portfolio  
0   1000000 100.76              1000000      USD  BondPortfolioForPnLCalc

In [394]:
for index, row in transactions.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=row['portfolio'],
        transaction_request=[
            lm.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["settlement_date"],
                units=row["quantity"],
                transaction_price=lm.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=lm.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            )
        ],
    )

# 4. Quotes

## 4.1 Book Quotes

In [338]:
bond_prices = pd.read_csv('data/bond_quotes_data.csv')
bond_prices.head(7)

client_internal        date  price currency
0    T_3_02/15/49  15/02/2019 100.11      USD
1    T_3_02/15/49  18/02/2019 100.11      USD
2    T_3_02/15/49  19/02/2019 100.42      USD
3    T_3_02/15/49  20/02/2019 100.09      USD
4    T_3_02/15/49  21/02/2019  99.02      USD
5    T_3_02/15/49  22/02/2019  99.67      USD
6    T_3_02/15/49  25/02/2019  99.46      USD

In [339]:

for index, row in bond_prices.iterrows():

    instrument_quotes = {
        "upsert_request": lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["client_internal"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=to_date(row["date"]),
            ),
            metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
            scale_factor=100            
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=scope, request_body=instrument_quotes
    )

# 5. Valuations

## 5.1 Create valuation recipe

In [415]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="bondValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Credit.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

In [416]:
def get_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="bondValuation"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Holding/default/Accrual": "Accrued Interest",
        },
        inplace=True,
    )

    return vals_df

## Day 1

At 10AM on day 1, once our bond has settled, we value our bond position with the SimpleStatic model and find that it is worth $1,006,662.46. This value includes about two months worth of accrued interest as the last coupon was on 2019-02-15 and we are now valuing it on 22/04/2019.

In [417]:
get_val("2019-04-22T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,006,909.03   T 3 02/15/49   T_3_02/15/49        100.15   

   Holding/default/Units  Accrued Interest PnL (1-day)  \
0           1,000,000.00          5,424.66        None   

                                  Aggregation/Errors  
0  [Failed to evaluate scripted task. Result was ...

## Day 2

On day 2, we run our valuation again and find that our accrued interest increased and the price has moved up. Our position is now worth $2,347.82 more than on day 1. Our PnL after 1 day is thus $2,347.82 on this position.

In [418]:
get_val("2019-04-23T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,009,256.85   T 3 02/15/49   T_3_02/15/49        100.38   

   Holding/default/Units  Accrued Interest  PnL (1-day)  
0           1,000,000.00          5,506.85     2,347.82

## Coupon date

One day before our coupon date, we see the accrued interest approaching the coupon value of $15,000. When the coupon date hits, the accrued interest will reset to zero. In our case, we want to add a cash position of $15,000 to our portfolio so that we end up with the correct value on the coupon date. This is where our cash flow management functions come in.

In [419]:
get_val("2019-08-14T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,234,013.27   T 3 02/15/49   T_3_02/15/49        121.92   

   Holding/default/Units  Accrued Interest  PnL (1-day)  
0           1,000,000.00         14,794.52    35,707.19

In [420]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,230,468.75   T 3 02/15/49   T_3_02/15/49        123.05   

   Holding/default/Units  Accrued Interest  PnL (1-day)  
0           1,000,000.00              0.00    -3,544.52

# 6. Cash Flows

## 6.1 Cash Ladder

If we take a look at the cash ladder for our portfolio on 2019-08-14, we can see the coupons that will have happened between 2019-08-14 and the time of writing this example case (2021-10-28).

In [421]:
cash_ladder = transaction_portfolio_api.get_portfolio_cash_ladder(scope=scope, code=portfolio_code, effective_at="2019-08-14T10:00:00Z", from_effective_at="2019-08-14T10:00:00Z", to_effective_at="2021-10-28T10:00:00Z")
lusid_response_to_data_frame(cash_ladder)

currency sub_holding_keys  records.0.effective_date  records.0.open  \
0      USD               {} 2019-08-15 00:00:00+00:00            0.00   

   records.0.activities.Coupon  records.0.close  records.1.effective_date  \
0                    14,876.71        14,876.71 2020-02-17 00:00:00+00:00   

   records.1.open  records.1.activities.Coupon  records.1.close  \
0       14,876.71                    15,277.12        30,153.83   

   records.2.effective_date  records.2.open  records.2.activities.Coupon  \
0 2020-08-17 00:00:00+00:00       30,153.83                    14,918.03   

   records.2.close  records.3.effective_date  records.3.open  \
0        45,071.86 2021-02-15 00:00:00+00:00       45,071.86   

   records.3.activities.Coupon  records.3.close  records.4.effective_date  \
0                    14,928.14        60,000.00 2021-08-16 00:00:00+00:00   

   records.4.open  records.4.activities.Coupon  records.4.close  
0       60,000.00                    14,958.90        74,958.90

We can also take a look and see all the future cash flows we can expect from this bond by running the same function, but setting the time frame from now until a few years in the future. In this example we are retrieving the expected cash flows from 2021-10-28 until 2025-10-28.

In [422]:
cash_ladder = transaction_portfolio_api.get_portfolio_cash_ladder(scope=scope, code=portfolio_code, effective_at="2021-10-28T10:00:00Z", from_effective_at="2021-10-28T10:00:00Z", to_effective_at="2025-10-28T10:00:00Z")
lusid_response_to_data_frame(cash_ladder)

currency sub_holding_keys  records.0.effective_date  records.0.open  \
0      USD               {} 2022-02-15 00:00:00+00:00            0.00   

   records.0.activities.Coupon  records.0.close  records.1.effective_date  \
0                    15,041.10        15,041.10 2022-08-15 00:00:00+00:00   

   records.1.open  records.1.activities.Coupon  records.1.close  \
0       15,041.10                    14,876.71        29,917.81   

   records.2.effective_date  records.2.open  records.2.activities.Coupon  \
0 2023-02-15 00:00:00+00:00       29,917.81                    15,123.29   

   records.2.close  records.3.effective_date  records.3.open  \
0        45,041.10 2023-08-15 00:00:00+00:00       45,041.10   

   records.3.activities.Coupon  records.3.close  records.4.effective_date  \
0                    14,876.71        59,917.81 2024-02-15 00:00:00+00:00   

   records.4.open  records.4.activities.Coupon  records.4.close  \
0       59,917.81                    15,113.18        75,030.99   

   records.5.effective_date  records.5.open  records.5.activities.Coupon  \
0 2024-08-15 00:00:00+00:00       75,030.99                    14,918.03   

   records.5.close  records.6.effective_date  records.6.open  \
0        89,949.02 2025-02-17 00:00:00+00:00       89,949.02   

   records.6.activities.Coupon  records.6.close  records.7.effective_date  \
0                    15,256.46       105,205.48 2025-08-15 00:00:00+00:00   

   records.7.open  records.7.activities.Coupon  records.7.close  
0      105,205.48                    14,712.33       119,917.81

## 6.2 Cash Flow table

Below we can see the cash flow table for our portfolio which contains our bond. We can see the coupons we are owed and the final principal repayment on the maturity date. We can proceed to upload some of these cash payments that have already happened to our portfolio so that we have a correct cash position. 

In [433]:
upsertable_cash_flows = transaction_portfolio_api.get_upsertable_portfolio_cash_flows(scope=scope, code=portfolio_code, effective_at="2021-08-14T10:00:00Z", window_start="2045-02-15T10:00:00Z", window_end="2049-02-15T10:00:00Z")
lusid_response_to_data_frame(upsertable_cash_flows)

transaction_id      type instrument_identifiers instrument_uid  \
0  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
1  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
2  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
3  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
4  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
5  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
6  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
7  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   
8  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   

           transaction_date           settlement_date        units  \
0 2045-08-15 00:00:00+00:00 2045-08-15 00:00:00+00:00    14,876.71   
1 2046-02-15 00:00:00+00:00 2046-02-15 00:00:00+00:00    15,123.29   
2 2046-08-15 00:00:00+00:00 2046-08-15 00:00:00+00:00    14,876.71   
3 2047-02-15 00:00:00+00:00 2047-02-15 00:00:00+00:00    15,123.29   
4 2047-08-15 00:00:00+00:00 2047-08-15 00:00:00+00:00    14,876.71   
5 2048-02-17 00:00:00+00:00 2048-02-17 00:00:00+00:00    15,277.12   
6 2048-08-17 00:00:00+00:00 2048-08-17 00:00:00+00:00    14,918.03   
7 2049-02-15 00:00:00+00:00 2049-02-15 00:00:00+00:00    14,928.14   
8 2049-02-15 00:00:00+00:00 2049-02-15 00:00:00+00:00 1,000,000.00   

   transaction_price.price transaction_price.type  total_consideration.amount  \
0                     1.00                  Price                   14,876.71   
1                     1.00                  Price                   15,123.29   
2                     1.00                  Price                   14,876.71   
3                     1.00                  Price                   15,123.29   
4                     1.00                  Price                   14,876.71   
5                     1.00                  Price                   15,277.12   
6                     1.00                  Price                   14,918.03   
7                     1.00                  Price                   14,928.14   
8                     1.00                  Price                1,000,000.00   

  total_consideration.currency  exchange_rate transaction_currency properties  \
0                          USD           1.00                  USD         {}   
1                          USD           1.00                  USD         {}   
2                          USD           1.00                  USD         {}   
3                          USD           1.00                  USD         {}   
4                          USD           1.00                  USD         {}   
5                          USD           1.00                  USD         {}   
6                          USD           1.00                  USD         {}   
7                          USD           1.00                  USD         {}   
8                          USD           1.00                  USD         {}   

  source            entry_date_time  
0         0001-01-01 00:00:00+00:00  
1         0001-01-01 00:00:00+00:00  
2         0001-01-01 00:00:00+00:00  
3         0001-01-01 00:00:00+00:00  
4         0001-01-01 00:00:00+00:00  
5         0001-01-01 00:00:00+00:00  
6         0001-01-01 00:00:00+00:00  
7         0001-01-01 00:00:00+00:00  
8         0001-01-01 00:00:00+00:00

In our valuation of the coupon date we did above, we saw the accrued interest for the coupon on 2019-08-15 reset. Now we will retrieve this cash flow and upsert it to LUSID so that in our valuation, we can see the bond and the cash position together.

In [426]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,230,468.75   T 3 02/15/49   T_3_02/15/49        123.05   

   Holding/default/Units  Accrued Interest  PnL (1-day)  
0           1,000,000.00              0.00    -3,544.52

In [427]:
upsertable_cash_flows = transaction_portfolio_api.get_upsertable_portfolio_cash_flows(scope=scope, code=portfolio_code, effective_at="2019-08-14T10:00:00Z", window_start="2019-08-14T10:00:00Z", window_end="2019-08-30T10:00:00Z")
lusid_response_to_data_frame(upsertable_cash_flows)

transaction_id      type instrument_identifiers instrument_uid  \
0  -LUID_0000BYLX-USD-Receive  CashFlow                     {}  LUID_0000BYLX   

           transaction_date           settlement_date     units  \
0 2019-08-15 00:00:00+00:00 2019-08-15 00:00:00+00:00 14,876.71   

   transaction_price.price transaction_price.type  total_consideration.amount  \
0                     1.00                  Price                   14,876.71   

  total_consideration.currency  exchange_rate transaction_currency properties  \
0                          USD           1.00                  USD         {}   

  source            entry_date_time  
0         0001-01-01 00:00:00+00:00

We want to book this cashflow into LUSID now so that we have an accurate cash position for our valuation. We have to make a few minor adjustments such as specifying which instrument we want to book these cash flows into. In our case, this is the USD currency identifier. We must also ensure the transaction ID's are unique, thus we tag on the date of the coupon to the transaction ID.

Once this is done, we upsert these cash flows to our portfolio.

In [428]:
for x in upsertable_cash_flows.values:
    x.instrument_identifiers = {"Instrument/default/Currency": "GBP"}
    x.transaction_id = x.transaction_id +'-' + str(x.transaction_date)

In [429]:
upsert_transactions = transaction_portfolio_api.upsert_transactions(
    scope=scope, 
    code=portfolio_code, 
    transaction_request=upsertable_cash_flows.values
    )

If we now want to see the valuation of our portfolio on the coupon date, we can see the bond position has diminished by the coupon amount and the cash position has increased by that amount.

In [430]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

Value InstrumentName ClientInternal  Quotes/Price  \
0 1,230,468.75   T 3 02/15/49   T_3_02/15/49        123.05   
1    14,876.71            USD           None           nan   

   Holding/default/Units  Accrued Interest  PnL (1-day)  
0           1,000,000.00              0.00    -3,544.52  
1              14,876.71              0.00         0.00